In [1]:
import mindspore
import mindspore.nn as nn
from mindspore import Tensor, load_checkpoint, load_param_into_net

import mindspore.dataset as ds
from mindspore import context
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy

import os, pickle

import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.common import dtype as mstype

import numpy as np

import matplotlib.pyplot as plt

import cv2

In [2]:
'''lenet'''

class LeNet5(nn.Cell):
    def __init__(self, num_class=10, channel=3):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(channel, 6, 5, pad_mode="pad")
        self.deconv1 = nn.Conv2dTranspose(6, 3, 5, pad_mode="pad")
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode="pad")
        self.fc1 = nn.Dense(16*5*5, 120)
        self.fc2 = nn.Dense(120, 84)
        self.fc3 = nn.Dense(84, num_class)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        
    def construct(self, x):
        #命名中间层输出
        self.conv1_output = self.relu(self.conv1(x))
        self.deconv1_output = self.deconv1(self.conv1_output)
        self.pool1_output = self.pool(self.conv1_output)
        self.conv2_output = self.relu(self.conv2(self.pool1_output))
        self.pool2_output = self.pool(self.conv2_output)
        x = self.flatten(self.pool2_output)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

network = LeNet5(10)

In [3]:
def get_data(datapath):
    cifar_ds = ds.Cifar10Dataset(datapath)
    return cifar_ds

def process_dataset(cifar_ds,batch_size =32,status="train"):
    '''
    ---- 定义算子 ----
    '''
    # 归一化
    rescale = 1.0 / 255.0
    # 平移
    shift = 0.0

    resize_op = CV.Resize((32, 32))
    rescale_op = CV.Rescale(rescale, shift)
    # 对于RGB三通道分别设定mean和std
    normalize_op = CV.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    if status == "train":
        # 随机裁剪
        random_crop_op = CV.RandomCrop([32, 32], [4, 4, 4, 4])
        # 随机翻转
        random_horizontal_op = CV.RandomHorizontalFlip()
    # 通道变化
    channel_swap_op = CV.HWC2CHW()
    # 类型变化
    typecast_op = C.TypeCast(mstype.int32)

    '''
    ---- 算子运算 ----
    '''
    cifar_ds = cifar_ds.map(input_columns="label", operations=typecast_op)
    if status == "train":
        cifar_ds = cifar_ds.map(input_columns="image", operations=random_crop_op)
        cifar_ds = cifar_ds.map(input_columns="image", operations=random_horizontal_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=resize_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=rescale_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=normalize_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=channel_swap_op)
    
    # shuffle
    cifar_ds = cifar_ds.shuffle(buffer_size=1000)
    # 切分数据集到batch_size
    cifar_ds = cifar_ds.batch(batch_size, drop_remainder=True)
    
    return cifar_ds

In [4]:
# 返回当前设备
device_target = mindspore.context.get_context('device_target')
# 确定图模型是否下沉到芯片上
dataset_sink_mode = True if device_target in ['Ascend','GPU'] else False
# 使用交叉熵函数作为损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
# 优化器为Adam
net_opt = nn.Adam(params=network.trainable_params(), learning_rate=0.001)

In [5]:
current_path = os.getcwd()
checkpoint_path = os.path.join(current_path, 'results/checkpoint_lenet_original-100_1562.ckpt')
load_checkpoint(checkpoint_path, net=network)

data_path = os.path.join(current_path, 'data/10-verify-bin')
batch_size=32
status="test"
# 生成测试数据集
cifar_ds = ds.Cifar10Dataset(data_path)
ds_eval = process_dataset(cifar_ds,batch_size=batch_size,status=status)
model = Model(network = network, loss_fn=net_loss,optimizer=net_opt, metrics={"Accuracy": Accuracy()})

res = model.eval(ds_eval, dataset_sink_mode=dataset_sink_mode)
# 评估测试集
print('test results:',res)

ValueError: The checkpoint file does not exist.

In [ ]:
rescale = 1.0 / 255.0
shift = 0.0

img = cv2.imread('0.jpg')
resize = CV.Resize((32, 32))
rescale = CV.Rescale(rescale, shift)
normalize = CV.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
channel_op = CV.HWC2CHW()

img = resize(img)
img = rescale(img)
img = normalize(img)
img = channel_op(img)

#添加一个维度batch_size
img = [img]
img = Tensor(img, mstype.float32)

input = img
output = network(input)

print(network.deconv1_output.shape)
layer_name = 'lenet5_deconv1_'
image_name = '0'
#conv1_output 1x6x28x28
features = network.deconv1_output

for i in range(features.shape[0]):
    feature = features[i, :, :, :]
    feature = np.transpose(feature.asnumpy(), [1, 2, 0])
    #归一化
    feature = (feature-np.amin(feature))/(np.amax(feature)-np.amin(feature))
    plt.imshow(feature)
plt.savefig('./featuremap/'+layer_name+image_name+'.png')
plt.show()

In [ ]:
for dataset in ['day_left', 'day_right', 'night_right']:
    prepath = 'data/GardensPoint/'+dataset+'/'
    images = [f for f in os.listdir(prepath) if f.endswith('.jpg')]
    images.sort()

    def preprocess(img):
        rescale = 1.0 / 255.0
        shift = 0.0
        resize = CV.Resize((32, 32))
        rescale = CV.Rescale(rescale, shift)
        normalize = CV.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        channel_op = CV.HWC2CHW()

        img = resize(img)
        img = rescale(img)
        img = normalize(img)
        img = channel_op(img)

        #添加一个维度batch_size
        #img = [img]
        #img = Tensor(img, mstype.float32)
        return img

    input = []
    for i in range(len(images)):
        image = cv2.imread(prepath+images[i])
        input.append(preprocess(image))
    input = np.array(input)
    input = Tensor(input, mstype.float32)

    output = network(input)
    with open('net_'+dataset+'_conv2', 'wb') as fw:
        #conv1(200, 6, 28, 28),将tensor转化为numpy数组
        pickle.dump(network.conv2_output.asnumpy(), fw)


In [ ]:
layer = 'conv2'
with open('net_day_left_'+layer, 'rb') as fr:
    Xdl = pickle.load(fr).reshape((200, -1))
    
with open('net_day_right_'+layer, 'rb') as fr:
    Xdr = pickle.load(fr).reshape((200, -1))
    
with open('net_night_right_'+layer, 'rb') as fr:
    Xnr = pickle.load(fr).reshape((200, -1))

Xdl /= np.sqrt(np.sum(np.power(Xdl, 2), axis=1)).reshape(200, 1)
Xdr /= np.sqrt(np.sum(np.power(Xdr, 2), axis=1)).reshape(200, 1)
Xnr /= np.sqrt(np.sum(np.power(Xnr, 2), axis=1)).reshape(200, 1)

#矩阵相乘
D_nr_dr = Xnr.dot(Xdr.T)
with open('D_nr_dr_'+layer, 'wb') as fw:
    pickle.dump(D_nr_dr, fw)
#plt.imshow(D_nr_dr)
#plt.colorbar()
#plt.title('night right vs day right')
#plt.savefig('conv1_consine_nr_dr.png', bbox_inches='tight')
#plt.show()

D_dl_dr = Xdl.dot(Xdr.T)
with open('D_dl_dr_'+layer, 'wb') as fw:
    pickle.dump(D_dl_dr, fw)
    
D_nr_dl = Xnr.dot(Xdl.T)
with open('D_nr_dl_'+layer, 'wb') as fw:
    pickle.dump(D_nr_dl, fw)

In [ ]:
#数据集每一个场景都有对应的ground truth匹配，所以没有true negatives
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

#true-positive region: 相差正负e帧
#论文中e取值1-5，根据数据集帧率
e = 3
#比率测试：最佳匹配与第二最佳匹配的余弦距离之比，如果大于r，则为positive
r_range = np.arange(1, 2, .001)

layers = ['conv2']
for layer in layers:
    
    with open('D_nr_dr_'+layer, 'rb') as f:
        D_nr_dr = pickle.load(f)

    with open('D_dl_dr_'+layer, 'rb') as f:
        D_dl_dr = pickle.load(f)
    
    with open('D_nr_dl_'+layer, 'rb') as f:
        D_nr_dl = pickle.load(f)

    #D_nr_dr：[200, 200]
    n = D_nr_dr.shape[0]
    AUC = []
    
    for D in [D_nr_dr, D_dl_dr, D_nr_dl]:
        TP = np.zeros_like(r_range)
        FP = np.zeros_like(r_range)
        FN = np.zeros_like(r_range)
        for i in range(len(r_range)):
            r = r_range[i]
            for row in range(n):
                #numpy.argsort返回数组值由小到大的索引值
                col2, col1 = np.argsort(D[row])[-2:]
                if (D[row, col1] / D[row, col2]) >= r:
                    
                    if (row <= col1+e) and (row >= col1-e):
                        TP[i] += 1
                    else:
                        FP[i] += 1
                else:
                    FN[i] += 1
        #保留有效指标       
        IX = (TP+FP) > 0
        R = TP[IX] / (TP[IX] + FN[IX])
        P = TP[IX] / (TP[IX] + FP[IX])
        AUC.append(np.trapz(P[::-1], R[::-1]))
        plt.plot(R, P)
        # F1 = 2 * (P*R) / (P+R)
        
    labels = ['AUC=%.3f (night-right vs day-right)' %AUC[0],
              'AUC=%.3f (day-left vs day-right)'    %AUC[1],
              'AUC=%.3f (night-right vs day-left)'  %AUC[2]]
    
    p = np.arange(.001, 1, .001)
    for f in np.arange(.1,1,.1):
        r = np.array([f*v/(2*v-f) if 2*v!=f else -1 for v in p])
        p_cut = p[np.logical_and(r>=0,r<=1)]
        r_cut = r[np.logical_and(r>=0,r<=1)]
        plt.plot(r_cut, p_cut, "--", color='gray')
        plt.annotate(r"$F_1=%.1f$" % f, xy=(r_cut[0], p_cut[0]),
            xytext=(.9, p_cut[0]), size="small", color="gray")

    plt.legend(labels, loc='lower left')
    plt.xlim([-0.01,1.01])
    plt.xlabel('Recall')
    plt.ylim([-0.01,1.01])
    plt.ylabel('Precision')
    plt.title('Precision-Recall curve for layer: {0}'.format(layer))
    plt.savefig('PR_'+layer+'.png', bbox_inches='tight')
    plt.show()
